# Person Detection

## PreTrain Model: yolo11n
<html>
    <img src="https://learnopencv.com/wp-content/uploads/2024/10/feature.gif" />
</html>

In [1]:
!pip install fiftyone
!pip install ultralytics
!pip install tensorboard

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
%load_ext tensorboard
# Enable TensorBoard logging
!yolo settings tensorboard=True

JSONDict("C:\Users\budik\AppData\Roaming\Ultralytics\settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "Z:\\BEKA\\2025\\AI\\Bootcamp\\Source\\Advanced CV\\Project 2\\Test\\datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "82ab06d11bd7f08b96129e1321931b9e33d6393056b2f4dd07d7ac2f5bcd8dab",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true,
  "openvino_msg": false
}
 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [3]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

import yaml
import matplotlib.pyplot as plt
import cv2

import shutil
import torch
import torch.nn as nn
from ultralytics import YOLO

from typing import Dict, Tuple

# Dataset

## Coco-2017
<table>
    <tr>
        <td width="50%"><img src="https://docs.cvat.ai/images/fiftyone_logo.png" /></td>
        <td width="50%"><img src="https://cocodataset.org/images/coco-logo.png" /></td>
    </tr>
</table>


<li>dataset: coco-2017</li>
<li>label_types: detections, the task is for "Person" detection (object detection)</li>
<li>classes: ["person"], downloaded dataset has person label</li>
<li>max_samples: train: 3000, validation: 500</li>
<li>seed: 42, for data consistency</li>

In [4]:
# Load train split
train_dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["detections"],
    classes=["person"],
    max_samples=3000,
    seed=42
)

# Load validation split
val_dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    classes=["person"],
    max_samples=500,
    seed=42
)

# Filter label person only
train_dataset = train_dataset.filter_labels("ground_truth", F("label") == "person")
val_dataset = val_dataset.filter_labels("ground_truth", F("label") == "person")


Found annotations at 'C:\Users\budik\fiftyone\coco-2017\raw\instances_train2017.json'
Sufficient images already downloaded
Existing download of split 'train' is sufficient
You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information
Loading existing dataset 'coco-2017-train-3000'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Found annotations at 'C:\Users\budik\fiftyone\coco-2017\raw\instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'coco-2017-validation-500'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [5]:
# Tag samples in the original datasets before merging
for sample in train_dataset:
    sample.tags.append("train")
    sample.save()

for sample in val_dataset:
    sample.tags.append("validation")
    sample.save()


In [6]:
dataset_name = "coco-2017-person-combined"

if fo.dataset_exists(dataset_name):
    print(f"Dataset '{dataset_name}' already exists. Deleting it to create a new one.")
    fo.delete_dataset(dataset_name)

print(f"Creating a new dataset '{dataset_name}'.")
combined_dataset = fo.Dataset(dataset_name)
combined_dataset.add_samples(train_dataset)
combined_dataset.add_samples(val_dataset)

Dataset 'coco-2017-person-combined' already exists. Deleting it to create a new one.
Creating a new dataset 'coco-2017-person-combined'.
 100% |███████████████| 3000/3000 [9.6s elapsed, 0s remaining, 306.0 samples/s]       
 100% |█████████████████| 500/500 [1.6s elapsed, 0s remaining, 317.9 samples/s]         


['68591382358aa6c0c0c0966e',
 '68591382358aa6c0c0c09693',
 '68591382358aa6c0c0c09694',
 '68591382358aa6c0c0c09695',
 '68591382358aa6c0c0c09696',
 '68591382358aa6c0c0c09697',
 '68591382358aa6c0c0c09698',
 '68591382358aa6c0c0c09699',
 '68591382358aa6c0c0c0969a',
 '68591382358aa6c0c0c097a8',
 '68591382358aa6c0c0c097a9',
 '68591382358aa6c0c0c097aa',
 '68591382358aa6c0c0c097ab',
 '68591382358aa6c0c0c097ac',
 '68591382358aa6c0c0c097ad',
 '68591382358aa6c0c0c097ae',
 '68591382358aa6c0c0c097af',
 '68591382358aa6c0c0c097b0',
 '68591382358aa6c0c0c097b1',
 '68591382358aa6c0c0c097b2',
 '68591382358aa6c0c0c097b3',
 '68591382358aa6c0c0c097b4',
 '68591382358aa6c0c0c097b5',
 '68591382358aa6c0c0c097b6',
 '68591382358aa6c0c0c097b7',
 '68591382358aa6c0c0c097b8',
 '68591382358aa6c0c0c097b9',
 '68591382358aa6c0c0c097ba',
 '68591382358aa6c0c0c097bb',
 '68591382358aa6c0c0c097bc',
 '68591382358aa6c0c0c097bd',
 '68591382358aa6c0c0c097be',
 '68591382358aa6c0c0c097bf',
 '68591382358aa6c0c0c097c0',
 '68591382358a

In [8]:
import os
# ----------------------
# 2. Export in YOLO Format
# ----------------------
export_dir = "./yolo_person_dataset"
shutil.rmtree(export_dir) # ensure clean dataset
split_map = {"train": "train", "validation": "val"}  # Map FiftyOne splits to YOLO splits

# Export filtered dataset
for split in split_map.keys():
    split_view = combined_dataset.match_tags(split)
    split_view.export(
        export_dir=os.path.join(export_dir, split_map[split]),
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="ground_truth",
        classes=["person"]  # Only export 'person' class
    )


 100% |███████████████| 3000/3000 [10.1s elapsed, 0s remaining, 197.7 samples/s]      
 100% |█████████████████| 500/500 [1.5s elapsed, 0s remaining, 363.8 samples/s]         


In [9]:
session = fo.launch_app(combined_dataset)


Could not connect session, trying again in 10 seconds



In [10]:

# ----------------------
# 3. Create dataset.yaml
# ----------------------
yolo_config = {
    "path": os.path.abspath(export_dir),
    "train": "train/images",
    "val": "val/images",
    "nc": 1,
    "names": ["person"]
}

with open(os.path.join(export_dir, "dataset.yaml"), "w") as f:
    yaml.dump(yolo_config, f)


# Model-1: PreTrained Yolo11n

In [11]:
def print_trainable_layers(model):
    print("Trainable layers:")
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f" - {name}")

def print_frozen_layers(model):
    print("Frozen layers:")
    for name, param in model.named_parameters():
        if not param.requires_grad:
            print(f" - {name}")



In [20]:
PROJECT = "runs/detect"
NAME = "person_detect_v1"

shutil.rmtree(os.path.join(PROJECT, NAME), ignore_errors=True)


In [21]:
model = YOLO("yolo11n.pt")  # Load pretrained model

print(model.model)

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
   

<html>
    <li>Backbone: Layer 0 - 10 --> After C2PSA</li>
    <li>Neck : Layer 11 - 22</li>
    <li>Head : Layer 23 --> Detect</li>
</html>

In [22]:
# Ensure that all layers are trainable
for name, param in model.model.named_parameters():
    param.requires_grad = True

print_frozen_layers(model.model)
print_trainable_layers(model.model)

Frozen layers:
Trainable layers:
 - model.0.conv.weight
 - model.0.bn.weight
 - model.0.bn.bias
 - model.1.conv.weight
 - model.1.bn.weight
 - model.1.bn.bias
 - model.2.cv1.conv.weight
 - model.2.cv1.bn.weight
 - model.2.cv1.bn.bias
 - model.2.cv2.conv.weight
 - model.2.cv2.bn.weight
 - model.2.cv2.bn.bias
 - model.2.m.0.cv1.conv.weight
 - model.2.m.0.cv1.bn.weight
 - model.2.m.0.cv1.bn.bias
 - model.2.m.0.cv2.conv.weight
 - model.2.m.0.cv2.bn.weight
 - model.2.m.0.cv2.bn.bias
 - model.3.conv.weight
 - model.3.bn.weight
 - model.3.bn.bias
 - model.4.cv1.conv.weight
 - model.4.cv1.bn.weight
 - model.4.cv1.bn.bias
 - model.4.cv2.conv.weight
 - model.4.cv2.bn.weight
 - model.4.cv2.bn.bias
 - model.4.m.0.cv1.conv.weight
 - model.4.m.0.cv1.bn.weight
 - model.4.m.0.cv1.bn.bias
 - model.4.m.0.cv2.conv.weight
 - model.4.m.0.cv2.bn.weight
 - model.4.m.0.cv2.bn.bias
 - model.5.conv.weight
 - model.5.bn.weight
 - model.5.bn.bias
 - model.6.cv1.conv.weight
 - model.6.cv1.bn.weight
 - model.6.cv1.

In [23]:
# Train with custom learning rate and optimizer
results = model.train(
    data=os.path.join(export_dir, "dataset.yaml"),
    epochs=50,
    batch=16,
    imgsz=640,
    optimizer="AdamW",
    lr0=0.001,              # Standard starting LR
    lrf=0.01,               # Final LR after decay
    momentum=0.9,
    weight_decay=0.0005,    
    warmup_epochs=3,        # Stabilize early training
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    cos_lr=True,            # Smooth cosine decay helps convergence
    device=0,
    workers=16,
    project=PROJECT,
    name=NAME
)


New https://pypi.org/project/ultralytics/8.3.158 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.152  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=./yolo_person_dataset\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.9, mosaic=1.0, multi_scale=False, name=person_detect_v1, nbs=64, n

train: Scanning Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 2\Train\yolo_person_dataset\train\labels\val.cache.

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access  (ping: 0.20.2 ms, read: 197.796.3 MB/s, size: 163.0 KB)


val: Scanning Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 2\Train\yolo_person_dataset\val\labels\val.cache... 5


...

      50/50      2.89G      1.002     0.7494      1.085         20        640: 100%|██████████| 188/188 [00:28<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:02

                   all        500       2041      0.771      0.612      0.704      0.454



50 epochs completed in 0.458 hours.
Optimizer stripped from runs\detect\person_detect_v1\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\person_detect_v1\weights\best.pt, 5.5MB

Validating runs\detect\person_detect_v1\weights\best.pt...
Ultralytics 8.3.152  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:03


                   all        500       2041      0.767      0.614      0.703      0.455
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\person_detect_v1


## Model v2

<html>
    <li>Yolo 11n</li>
    <li>Freeze Backbone and Neck</li>
</html>

In [16]:
PROJECT = "runs/detect"
NAME = "person_detect_v2"

shutil.rmtree(os.path.join(PROJECT, NAME), ignore_errors=True)


In [17]:
model = YOLO("yolo11n.pt")  # Load pretrained model

print(model.model)

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
   

<html>
    <li>Backbone: Layer 0 - 10 --> After C2PSA</li>
    <li>Neck : Layer 11 - 22</li>
    <li>Head : Layer 23 --> Detect</li>
</html>

In [18]:
# Freeze Backbone (Feature Extraction)
for name, param in model.model.named_parameters():
    if any(name.startswith(f"model.{i}.") for i in range(10)):
        param.requires_grad = False
    else:
        param.requires_grad = True

print_frozen_layers(model.model)
print_trainable_layers(model.model)

Frozen layers:
 - model.0.conv.weight
 - model.0.bn.weight
 - model.0.bn.bias
 - model.1.conv.weight
 - model.1.bn.weight
 - model.1.bn.bias
 - model.2.cv1.conv.weight
 - model.2.cv1.bn.weight
 - model.2.cv1.bn.bias
 - model.2.cv2.conv.weight
 - model.2.cv2.bn.weight
 - model.2.cv2.bn.bias
 - model.2.m.0.cv1.conv.weight
 - model.2.m.0.cv1.bn.weight
 - model.2.m.0.cv1.bn.bias
 - model.2.m.0.cv2.conv.weight
 - model.2.m.0.cv2.bn.weight
 - model.2.m.0.cv2.bn.bias
 - model.3.conv.weight
 - model.3.bn.weight
 - model.3.bn.bias
 - model.4.cv1.conv.weight
 - model.4.cv1.bn.weight
 - model.4.cv1.bn.bias
 - model.4.cv2.conv.weight
 - model.4.cv2.bn.weight
 - model.4.cv2.bn.bias
 - model.4.m.0.cv1.conv.weight
 - model.4.m.0.cv1.bn.weight
 - model.4.m.0.cv1.bn.bias
 - model.4.m.0.cv2.conv.weight
 - model.4.m.0.cv2.bn.weight
 - model.4.m.0.cv2.bn.bias
 - model.5.conv.weight
 - model.5.bn.weight
 - model.5.bn.bias
 - model.6.cv1.conv.weight
 - model.6.cv1.bn.weight
 - model.6.cv1.bn.bias
 - model.6

In [19]:
# Train with custom learning rate and optimizer
results = model.train(
    data=os.path.join(export_dir, "dataset.yaml"),
    epochs=50,
    batch=16,
    imgsz=640,
    optimizer="AdamW",
    lr0=0.0001,             # Lower initial LR
    lrf=0.01,               # Final LR after decay
    momentum=0.9,
    weight_decay=0.0005,    
    warmup_epochs=3,        # Stabilize early training
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    cos_lr=True,            # Smooth cosine decay helps convergence
    device=0,
    workers=16,
    project=PROJECT,
    name=NAME
)

New https://pypi.org/project/ultralytics/8.3.158 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.152  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=./yolo_person_dataset\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.9, mosaic=1.0, multi_scale=False, name=person_detect_v2, nbs=64, 

train: Scanning Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 2\Train\yolo_person_dataset\train\labels\val.cache.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.1 ms, read: 165.6138.9 MB/s, size: 163.0 KB)


val: Scanning Z:\BEKA\2025\AI\Bootcamp\Source\Advanced CV\Project 2\Train\yolo_person_dataset\val\labels\val.cache... 5


Plotting labels to runs\detect\person_detect_v2\labels.jpg... 
optimizer: AdamW(lr=0.0001, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 16 dataloader workers
Logging results to runs\detect\person_detect_v2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      2.95G     0.9451     0.7195      1.034         20        640: 100%|██████████| 188/188 [00:27<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:02

                   all        500       2041      0.787      0.633      0.732      0.482



50 epochs completed in 0.451 hours.
Optimizer stripped from runs\detect\person_detect_v2\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\person_detect_v2\weights\best.pt, 5.5MB

Validating runs\detect\person_detect_v2\weights\best.pt...
Ultralytics 8.3.152  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:03


                   all        500       2041      0.779       0.65      0.741       0.49
Speed: 0.2ms preprocess, 1.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\person_detect_v2
